<a href="https://colab.research.google.com/github/uoahy-6uoas/HG-Machine/blob/4-1-edit/hg4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**로지스틱 회귀**
> **로지스틱 회귀**는 수학을 사용하여 두 데이터 요인 간의 관계를 찾는 데이터 분석 기법 *AWS*



In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
pd.unique(fish['Species']) #fish는 데이터프레임 타입

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [3]:
# kn을 활용해 타깃의 species를 확률로 나타내기 위해 나머지 열을 input으로
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() #ndarray형식
fish_target = fish['Species'].to_numpy() #Species가 2차원 배열이 되지 않게 주의!

In [4]:
#갈라쇼
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target)

In [5]:
#전처리 근데 이제 스케일링
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [6]:
#kn
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()
kn.fit(train_scaled, train_target)
kn.score(train_scaled , train_target), kn.score(test_scaled, test_target)

(0.8571428571428571, 0.75)

> **다중분류**(*multi-class classification*)란 target에 2개 이상의 클래스가 포함된 문제

In [7]:
#과소적합이지만 일단 넘겨..
#sklearn에서는 자동으로 알파벳순으로 바뀜
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [8]:
import numpy as np
proba = kn.predict_proba(test_scaled[:10]) #처음 10개의 분류 확률을 araboza
print(kn.classes_)
print(np.round(proba, decimals=4)) #소수4번째까지
print(kn.predict(test_scaled[:10])) #확률이 높은 클래스로 예측함을 볼 수 있음

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[1.  0.  0.  0.  0.  0.  0. ]
 [1.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.4 0.6 0.  0.  0. ]
 [0.6 0.4 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.8 0.  0.  0.  0.2]
 [1.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.8 0.  0.2 0.  0. ]
 [0.  0.  0.6 0.4 0.  0.  0. ]]
['Bream' 'Bream' 'Pike' 'Bream' 'Smelt' 'Smelt' 'Perch' 'Bream' 'Perch'
 'Perch']


In [9]:
distances, indexes = kn.kneighbors(test_scaled[1:2]) #1번 인덱스가 왜 저런 확률로 나왔는지 확인해보자
train_target[indexes] #1번 인덱스의 이웃
# kneighbors()의 입력은 2차원배열, 아웃도 2차원배열

array([['Bream', 'Bream', 'Bream', 'Bream', 'Bream']], dtype=object)

확률을 추출하기 위해 kn을 사용했지만, k의 개수가 5개라 확률로 표현할 수 있는 범위가 제한적임 따라서 '**로지스틱 회귀**' 쓸 거
<br>
<br>
$ z = a \times (weight) + b \times (length) + c \times (diagonal) + d \times (height) + e \times (width) + f $ <br>
$ \phi = {1 \over 1+e^{-z}}$ : *sigmoid function* <br>
선형회귀와 마찬가지로 각 species마다의 파라미터들(a~f)을 학습함, phi값 0.5를 기준으로 긴지 아닌지 판단(0.5일때 sklearn은 아닌거로 판단)

로지스틱 회귀는 반복을 통해 학습시킴 <br>
LogisticRegression클래스의 max_iter매개변수에서 반복값 변경가능(defalut:100회)<br>
<br>
ridge회귀와 같이 계수의 n제곱을 규제함 (L2규제, C(alpha와 비슷)가 커지면 규제는 작아짐)<br>


In [10]:
#이제 로지스틱 다중분류 해보자
from sklearn.linear_model import LogisticRegression #리니어 모델이네??
lr = LogisticRegression(C=20,max_iter=1000)
lr.fit(train_scaled, train_target)
lr.score(train_scaled, train_target), lr.score(test_scaled, test_target)

(0.9495798319327731, 0.875)

In [11]:
#또 과소적합이지만 일단 넘어가..지금 중요한 건 로지스틱 확률 확인해 보는 것!
proba = lr.predict_proba(test_scaled[:10])
print(lr.classes_)
print(np.round(proba, decimals=3))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.982 0.005 0.    0.    0.    0.    0.013]
 [0.999 0.    0.    0.    0.    0.    0.001]
 [0.    0.    0.082 0.852 0.06  0.    0.006]
 [0.689 0.247 0.    0.    0.026 0.    0.038]
 [0.    0.003 0.049 0.    0.002 0.947 0.   ]
 [0.    0.002 0.055 0.    0.001 0.943 0.   ]
 [0.    0.    0.909 0.004 0.066 0.    0.021]
 [0.999 0.    0.    0.    0.    0.    0.001]
 [0.    0.065 0.446 0.001 0.413 0.045 0.03 ]
 [0.    0.    0.031 0.904 0.049 0.01  0.005]]


In [12]:
#(a~e), f shape확인
lr.coef_.shape, lr.intercept_.shape # species 7개 마다 파라미터가 존재함을 알 수 있음

((7, 5), (7,))

따라서 로지스틱 회귀시 서로다른 파라미터 집합을 분류개수 만큼 만들어 하나씩 대입해 z값을 뽑아냄<br>
그런데 확률이니 합이 1이 되어야 함 --> **소프트맥스**
> The **softmax** function is a function that turns a vector of K real values into a vector of K real values that sum to 1. *DeepAI*

*소프트맥스*<br>
$e_{sum} = e^{z1} + e^{z2}+ ... $<br>
$s1 = {e^{z1} \over e_{sum}} ... $<br>
proba 출력 = [[s1,s2, ... ]]<br>
*sklearn은 자동으로 소프트맥스 계산해 줌.*

In [13]:
#z값 표시하기
decision = lr.decision_function(test_scaled[:10])
print(np.round(decision,decimals=3))
print(type(decision))

[[ 1.0146e+01  4.7770e+00 -3.8390e+00 -2.0180e+00  2.2800e-01 -1.5118e+01
   5.8250e+00]
 [ 1.4622e+01  3.6490e+00 -4.3160e+00  1.3000e-01 -1.0690e+00 -2.0549e+01
   7.5330e+00]
 [-4.3360e+00 -8.3310e+00  3.7140e+00  6.0520e+00  3.4000e+00 -1.5300e+00
   1.0310e+00]
 [ 6.3670e+00  5.3380e+00 -3.3920e+00 -3.9200e+00  3.0920e+00 -1.0951e+01
   3.4650e+00]
 [-1.0882e+01  1.6400e+00  4.5860e+00 -3.7550e+00  1.2950e+00  7.5530e+00
  -4.3700e-01]
 [-1.2330e+01  2.1400e+00  5.6570e+00 -4.7560e+00  1.1870e+00  8.5030e+00
  -4.0200e-01]
 [-3.3060e+00 -2.2370e+00  5.4960e+00  5.0000e-03  2.8750e+00 -4.5400e+00
   1.7060e+00]
 [ 1.3168e+01  5.2210e+00 -5.4560e+00 -1.7280e+00  6.9000e-01 -1.8005e+01
   6.1100e+00]
 [-5.8770e+00  1.2770e+00  3.1980e+00 -3.1280e+00  3.1220e+00  9.0600e-01
   5.0200e-01]
 [-4.2700e+00 -6.3780e+00  1.9600e+00  5.3250e+00  2.4140e+00  8.5000e-01
   9.9000e-02]]
<class 'numpy.ndarray'>


In [14]:
#scipy로 소프트맥스 표시하기
from scipy.special import softmax
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.982 0.005 0.    0.    0.    0.    0.013]
 [0.999 0.    0.    0.    0.    0.    0.001]
 [0.    0.    0.082 0.852 0.06  0.    0.006]
 [0.689 0.247 0.    0.    0.026 0.    0.038]
 [0.    0.003 0.049 0.    0.002 0.947 0.   ]
 [0.    0.002 0.055 0.    0.001 0.943 0.   ]
 [0.    0.    0.909 0.004 0.066 0.    0.021]
 [0.999 0.    0.    0.    0.    0.    0.001]
 [0.    0.065 0.446 0.001 0.413 0.045 0.03 ]
 [0.    0.    0.031 0.904 0.049 0.01  0.005]]


확인문제
1. 2개보다 많은 클래스가 있는 분류문제를 다중분류라고 함
2. 로지스틱 회귀가 이진 분류에서 확률을 출력하기 위해 사용하는 함수는 시그모이드 함수
3. decision_function() 메서드의 출력(z)이 0일 때, 시그모이드 함수 값($\phi$)은 ${1\over2}=0.5$